In [ ]:
# Cell 1: (Unchanged) Install from GitHub for preview support
!pip install -q git+https://github.com/googleapis/python-genai.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Cell 2: Updated imports (add types)
from google import genai
from google.genai import types  # Added for Tool and FileSearch
import time
import os
from google.colab import userdata
from IPython.display import Markdown

# Set API key as environment variable from Colab secrets
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

# Create client (picks up GEMINI_API_KEY automatically)
client = genai.Client()
print("Client created successfully!")

Client created successfully!


In [ ]:
# # Cell 3: Create a sample file (or upload your own, e.g., 'my_document.pdf')
# # For demo, create a simple text file
# sample_content = """
# Gemini File Search is a fully managed RAG system.
# It handles uploading, chunking, embedding, and retrieval automatically.
# Supports formats like TXT, PDF, DOCX, and more.
# Pricing: $0.15 per 1M tokens for indexing; storage and queries free.
# """

# with open('sample.txt', 'w') as f:
#     f.write(sample_content)

# print("Sample file created: sample.txt")

Sample file created: sample.txt


In [ ]:
# Cell 4: (Minor tweak for polling; unchanged otherwise)
# Create the store
file_search_store = client.file_search_stores.create(
    config={'display_name': 'my-file-search-store'}
)
print(f"Created store: {file_search_store.name}")

# Upload and import the file
operation = client.file_search_stores.upload_to_file_search_store(
    file='/content/Profile.pdf',
    file_search_store_name=file_search_store.name,
    config={'display_name': 'Sample Document'}
)

# Wait for completion (poll the operation)
while not operation.done:
    print("Uploading and indexing... (this may take a minute)")
    time.sleep(5)
    operation = client.operations.get(operation)  # Docs: pass operation object

print("Upload and indexing complete!")

Created store: fileSearchStores/myfilesearchstore-w6sbxr4fguzv
Uploading and indexing... (this may take a minute)
Upload and indexing complete!


In [ ]:
# Cell 5: Updated query with proper types config and safe citations access
model = "gemini-2.5-flash"  # Latest supported

# Proper tool config per docs/GitHub
config = types.GenerateContentConfig(
    tools=[
        types.Tool(
            file_search=types.FileSearch(
                file_search_store_names=[file_search_store.name]
            )
        )
    ]
)

response = client.models.generate_content(
    model=model,
    contents="Who is Lakshmi narayana. Summarize his career",
    config=config
)

# Display response with Markdown
display(Markdown(response.text))

# Safe: Inspect and print citations
if response.candidates and len(response.candidates) > 0:
    candidate = response.candidates[0]
    if hasattr(candidate, 'grounding_metadata') and candidate.grounding_metadata:
        grounding = candidate.grounding_metadata
        print(f"\nFull Grounding Metadata: {grounding}")  # Inspect structure

        if hasattr(grounding, 'file_search') and grounding.file_search:
            print("\nCitations:")
            if hasattr(grounding.file_search, 'citations') and grounding.file_search.citations:
                for citation in grounding.file_search.citations:
                    print(f"- {getattr(citation, 'display_name', 'N/A')} (URI: {getattr(citation, 'uri', 'N/A')})")
            else:
                print("No citations found (but grounding occurred).")
        else:
            print("Grounding metadata present, but no 'file_search' attribute (tool may not have triggered fully).")
    else:
        print("No grounding metadata in response.")
else:
    print("No candidates in response.")

Lakshmi Narayana U is a digital business leader, AI consultant, and technologist with over 25 years of experience, primarily in internet, media, and entertainment sectors. He has held various senior and middle management positions, focusing on digital transformation and AI-driven innovations in content, business strategy, and technology.

His career highlights include:
*   **Head of Digital Media at Rachana Television Pvt. Ltd. (August 2015 - Present)**: He established and scaled the digital operations for NTV, Bhakthi TV, and Vanitha TV, including YouTube, OTT platforms, social media, and mobile apps. He is also exploring Generative AI for content repurposing and audience engagement.
*   **Founder of Movisvami (January 2022 - Present)**: He created a platform that merges cinema with business insights and developed Movisvami GPT, an AI assistant connecting movie themes to real-world business and management lessons. He also authored "Directing Business: Management Lessons from Movie Dreamers".
*   **Business Head at Geetha Arts Digital (February 2010 - July 2015)**: He developed online, digital, and mobile distribution channels, managed partnerships with major players like Amazon, iTunes, Netflix, and YouTube, and launched internet and mobile properties.
*   **Business Head at Cinematica Digitals Pvt. Ltd. (June 2011 - February 2013)**: He managed critical growth phases and set up processes for the company, which is a market leader in digital cinema solutions in Andhra Pradesh.
*   **Business Head at Silverscope Media and Entertainment P.Ltd (November 2010 - November 2011)**: He was responsible for the overall management of the company, defining its strategy and business model, and overseeing the sale of a majority stake.
*   **Bid Director at Tech Mahindra (formerly Mahindra Satyam) (August 2008 - August 2009)**: He managed bids for large strategic deals, driving response efforts for deals exceeding $100 million USD.
*   **Practice Head at Mahindra Satyam BPO (March 2008 - August 2008)**: He drove strategy alignment and execution, headed the PMO for inorganic growth accounts, and managed business performance initiatives.
*   **Business Unit Head at Catalytic Software (October 2006 - February 2008)**: He led the Web 2.0 consulting practice, managing consulting and delivery projects in Web 2.0 and new media.
*   **Founder and CEO of Arudra Interactive P.Ltd (2002 - February 2007)**: He provided project management solutions and customized products for the entertainment vertical, conceptualized and executed television and movie projects, and introduced Computer Aided Process Management (CAPM) for the entertainment industry with the brand SHOWIZ.
*   **Director at Vortex Technologies (2004 - September 2006)**: He was a member of the senior executive team and board of directors, involved in consulting, delivery, and business development.
*   **Vice President at Lanco Global Systems (April 1999 - November 2001)**: He established Lanco's IT arm, advised the board on e-business strategy, and developed and launched portals and related marketing efforts. He also headed the e-Solutions division with teams in the US, UK, and India.
*   **IT Consultant (January 1997 - March 1999)**: In the USA, he worked as a software consultant with companies primarily in Multimedia, Internet, and Groupware systems.

Lakshmi Narayana holds a B.E/B.Tech in Mechanical Engineering from Osmania University (1990-1994). He is skilled in Large Language Models (LLM), Agentic AI, and Multi-agent Systems.


Full Grounding Metadata: google_maps_widget_context_token=None grounding_chunks=[GroundingChunk(
  retrieved_context=GroundingChunkRetrievedContext(
    text="""--- PAGE 1 ---

Contact

lakshyindy@gmail.com

www.linkedin.com/in/laksh

(LinkedIn)

Top Skills

Large Language Models (LLM)
 Agentic Al

Multi-agent Systems

Certifications

Hands-On PyTorch Machine

Learning

Artificial Intelligence Foundations:
 Machine Learning

Artificial Intelligence Foundations:
 Thinking Machines

Artificial Intelligence Foundations:
 Neural Networks

Machine Learning Foundations:
 Linear Algebra

Lakshmi narayana U

Digital Business Leader | Al Consultant | Technologist
 Hyderabad, Telangana, India

Summary

Lakshmi Narayana is a digital business leader, Al consultant, and
 technologist with a passion for media and storytelling.

With 25+ years of experience across various countries, he has held
 senior and middle management roles, driving digital transformation
 in internet, media, and entertainment

In [ ]:
# Cell 5: Enhanced query with proper config and chunk-based "citations" printing
model = "gemini-2.5-flash"

# Proper tool config (unchanged)
config = types.GenerateContentConfig(
    tools=[
        types.Tool(
            file_search=types.FileSearch(
                file_search_store_names=[file_search_store.name]
            )
        )
    ]
)

response = client.models.generate_content(
    model=model,
    contents="Who is Lakshmi narayana. Summarize his career",
    config=config
)

# Display response with Markdown
display(Markdown(response.text))

# Enhanced: Print grounding as citations using chunks/supports
if response.candidates and len(response.candidates) > 0:
    candidate = response.candidates[0]
    if hasattr(candidate, 'grounding_metadata') and candidate.grounding_metadata:
        grounding = candidate.grounding_metadata
        print(f"\nFull Grounding Metadata: {grounding}")  # For debugging (remove if clean)

        if grounding.grounding_chunks:
            print("\n📄 Grounded Documents & Excerpts:")
            for i, chunk in enumerate(grounding.grounding_chunks):
                print(f"  {i+1}. **{chunk.retrieved_context.title}**")
                print(f"     URI: {getattr(chunk.retrieved_context, 'uri', 'N/A')}")
                print(f"     Excerpt: {chunk.retrieved_context.text[:200]}...")  # Truncate for brevity
                print()

            if grounding.grounding_supports:
                print("🔗 Grounded Segments in Response:")
                for j, support in enumerate(grounding.grounding_supports):
                    chunk_indices = support.grounding_chunk_indices
                    segment_text = support.segment.text[:100] + "..." if len(support.segment.text) > 100 else support.segment.text
                    print(f"  {j+1}. Supports chunk(s): {chunk_indices} | Segment: '{segment_text}' (chars {support.segment.start_index}-{support.segment.end_index})")
        else:
            print("No grounding chunks found.")
    else:
        print("No grounding metadata in response.")
else:
    print("No candidates in response.")

Lakshmi Narayana U is a digital business leader, AI consultant, and technologist from Hyderabad, Telangana, India, with over 25 years of experience in driving digital transformation within the internet, media, and entertainment sectors. He is also an entrepreneur who explores AI-driven innovations in content, business strategy, and technology. His top skills include Large Language Models (LLM), Agentic AI, and Multi-agent Systems.

His career highlights include:
*   **Rachana Television Pvt. Ltd.** He has been the Head of Digital Media since August 2015, where he established and scaled the digital operations for NTV, Bhakthi TV, and Vanitha TV, including YouTube, OTT platforms, social media, mobile apps, and community platforms. He is also exploring Generative AI for content repurposing and audience engagement.
*   **Movisvami.** As the Founder since January 2022, he created a platform that merges cinema and business insights. He developed Movisvami GPT, an AI assistant connecting movie themes with real-world business and management lessons, and authored "Directing Business: Management Lessons from Movie Dreamers".
*   **Geetha Arts Digital.** From February 2010 to July 2015, he served as Business Head, focusing on creating online, digital, and mobile distribution channels, managing partnerships with major players like Amazon, iTunes, Netflix, and YouTube, and launching internet and mobile properties.
*   **Cinematica Digitals Pvt. Ltd.** From June 2011 to February 2013, he was the Business Head, responsible for managing critical growth phases and setting up processes for the company, which is a key partner for Real Image digital cinema solutions in Andhra Pradesh.
*   **Silverscope Media and Entertainment P.Ltd.** As Business Head from November 2010 to November 2011, he managed the company's overall strategy, business model, and market orientation, and oversaw a management change and the sale of a majority stake.
*   **Tech Mahindra (formerly Mahindra Satyam).** He held roles as Bid Director from August 2008 to August 2009, managing large strategic deals, and as Practice Head for Mahindra Satyam BPO from March 2008 to August 2008, driving strategy alignment and execution.
*   **Catalytic Software.** From October 2006 to February 2008, he was a Business Unit Head, leading the Web 2.0 consulting practice and managing consulting and delivery projects.
*   **Arudra Interactive P.Ltd.** As Founder and CEO from 2002 to February 2007, he provided project management solutions for the entertainment industry, conceptualized and executed television and movie projects, and introduced Computer Aided Process Management (CAPM) with the brand SHOWIZ.
*   **Vortex Technologies.** He served as Director from 2004 to September 2006, where he was part of the senior executive team, involved in consulting, delivery, and business development, and helped position products in business performance management.
*   **Lanco Global Systems.** From April 1999 to November 2001, he was Vice President, establishing Lanco's IT arm, advising on e-business strategy, and heading the e-Solutions division across the US, UK, and India.
*   **IT Consultant (Self-employed).** From January 1997 to March 1999, he worked as a software consultant in the USA, primarily in Multimedia, Internet, and Groupware systems.

He holds a B.E/B.Tech in Mechanical Engineering from Osmania University (1990-1994).


Full Grounding Metadata: google_maps_widget_context_token=None grounding_chunks=[GroundingChunk(
  retrieved_context=GroundingChunkRetrievedContext(
    text="""--- PAGE 1 ---

Contact

lakshyindy@gmail.com

www.linkedin.com/in/laksh

(LinkedIn)

Top Skills

Large Language Models (LLM)
 Agentic Al

Multi-agent Systems

Certifications

Hands-On PyTorch Machine

Learning

Artificial Intelligence Foundations:
 Machine Learning

Artificial Intelligence Foundations:
 Thinking Machines

Artificial Intelligence Foundations:
 Neural Networks

Machine Learning Foundations:
 Linear Algebra

Lakshmi narayana U

Digital Business Leader | Al Consultant | Technologist
 Hyderabad, Telangana, India

Summary

Lakshmi Narayana is a digital business leader, Al consultant, and
 technologist with a passion for media and storytelling.

With 25+ years of experience across various countries, he has held
 senior and middle management roles, driving digital transformation
 in internet, media, and entertainment

In [ ]:
# Cell 5: Polished version with URI fallback, Markdown citations, and new query example
model = "gemini-2.5-flash"

config = types.GenerateContentConfig(
    tools=[
        types.Tool(
            file_search=types.FileSearch(
                file_search_store_names=[file_search_store.name]
            )
        )
    ]
)

# Try a slightly different query to demo variation
response = client.models.generate_content(
    model=model,
    contents="Who is Lakshmi narayana. Summarize his career",
    config=config
)

# Display response with Markdown
display(Markdown(response.text))

# Polished: Print grounding as formatted citations
if response.candidates and len(response.candidates) > 0:
    candidate = response.candidates[0]
    if hasattr(candidate, 'grounding_metadata') and candidate.grounding_metadata:
        grounding = candidate.grounding_metadata

        # Collect unique chunks for citations
        chunks = grounding.grounding_chunks if grounding.grounding_chunks else []
        if chunks:
            print("\n📚 **Citations from Your Documents** (Grounded Sources):")
            seen_chunks = set()  # Avoid duplicates
            for chunk in chunks:
                if id(chunk) in seen_chunks:
                    continue
                seen_chunks.add(id(chunk))

                title = getattr(chunk.retrieved_context, 'title', 'Untitled')
                uri = getattr(chunk.retrieved_context, 'uri', None)
                if uri is None:
                    # Fallback: Construct from store (replace with real doc ID if known)
                    uri = f"{file_search_store.name}/documents/sample-doc-fallback"

                excerpt = chunk.retrieved_context.text[:150].strip() + "..." if len(chunk.retrieved_context.text) > 150 else chunk.retrieved_context.text.strip()

                print(f"\n[{title}]({uri})  \n**Excerpt**: {excerpt}")
                print(f"**Relevance**: High (full chunk retrieved for semantic match)")

            # Segments as inline response links
            if grounding.grounding_supports:
                print(f"\n🔗 **Grounded Parts of This Response** (Exact Matches):")
                for j, support in enumerate(grounding.grounding_supports):
                    chunk_idx = support.grounding_chunk_indices[0] if support.grounding_chunk_indices else 'N/A'
                    segment = support.segment.text.strip()
                    start, end = support.segment.start_index, support.segment.end_index
                    truncated = segment[:80] + "..." if len(segment) > 80 else segment
                    print(f"  {j+1}. **Chunk {chunk_idx}** → '{truncated}' (positions {start}-{end})")
        else:
            print("\nNo grounding chunks (pure generation—try a more file-specific query).")
    else:
        print("No grounding metadata.")
else:
    print("No response candidates.")

Lakshmi Narayana U. is a digital business leader, AI consultant, and technologist from Hyderabad, Telangana, India, with over 25 years of experience, primarily in internet, media, and entertainment. He is passionate about media and storytelling and has a keen interest in AI-driven innovations.

His career highlights include:
*   **Rachana Television Pvt. Ltd.** (August 2015 - Present): Head of Digital Media, where he established and scaled the digital operations for NTV, Bhakthi TV, and Vanitha TV, and explores Generative AI for content and audience engagement.
*   **Movisvami** (January 2022 - Present): Founder of a platform that merges cinema and business insights. He also developed Movisvami GPT, an AI assistant, and authored "Directing Business: Management Lessons from Movie Dreamers."
*   **Geetha Arts Digital** (February 2010 - July 2015): As Business Head, he focused on online, digital, and mobile distribution, managed partnerships with major platforms like Amazon, iTunes, Netflix, and YouTube, and drove new initiatives in overseas markets.
*   **Cinematica Digitals Pvt. Ltd.** (June 2011 - February 2013): Served as Business Head, managing critical growth phases and aligning processes for the company, which is a partner of Real Image digital cinema solution (QUBE).
*   **Silverscope Media and Entertainment P.Ltd** (November 2010 - November 2011): Held the Business Head position, overseeing company management, strategy, business model, and market orientation. He also guided the company through management changes and a majority stake sale.
*   **Tech Mahindra (formerly Mahindra Satyam)** (August 2008 - August 2009): Worked as Bid Director, managing bids for large strategic deals exceeding $100 million USD.
*   **Mahindra Satyam BPO** (March 2008 - August 2008): As Practice Head, he focused on strategy alignment and execution, managed PMO for inorganic growth accounts, and oversaw business performance management initiatives.
*   **Catalytic Software** (October 2006 - February 2008): Headed the Web 2.0 consulting practice, managing projects and driving the practice to handle consulting and delivery in Web 2.0 and new media.
*   **Arudra Interactive P.Ltd** (2002 - February 2007): As Founder and CEO, he provided project management solutions and customized products for the entertainment vertical, introduced Computer Aided Process Management (CAPM) with the brand SHOWIZ, and developed internet strategies for NGOs.
*   **Vortex Technologies** (2004 - September 2006): Served as a Director and senior executive, involved in consulting, delivery, and business development, and positioning products in business performance management.
*   **Lanco Global Systems** (April 1999 - November 2001): As Vice President, he established Lanco's IT arm, advised the board on e-business strategy, and launched portals and related marketing efforts. He also headed the e-Solutions division with international teams.
*   **IT Consultant (Self-employed)** (January 1997 - March 1999): Worked as a software consultant in the USA with various companies, primarily in Multimedia, Internet, and Groupware systems.

He holds a B.E/B.Tech in Mechanical Engineering from Osmania University (1990-1994).


📚 **Citations from Your Documents** (Grounded Sources):

[Sample Document](fileSearchStores/myfilesearchstore-w6sbxr4fguzv/documents/sample-doc-fallback)  
**Excerpt**: --- PAGE 1 ---

Contact

lakshyindy@gmail.com

www.linkedin.com/in/laksh

(LinkedIn)

Top Skills

Large Language Models (LLM)
 Agentic Al

Multi-agent...
**Relevance**: High (full chunk retrieved for semantic match)

[Sample Document](fileSearchStores/myfilesearchstore-w6sbxr4fguzv/documents/sample-doc-fallback)  
**Excerpt**: assistant that connects movie
 themes with real-world business and management lessons.

- Authored Directing Business: Management Lessons from Movie D...
**Relevance**: High (full chunk retrieved for semantic match)

[Sample Document](fileSearchStores/myfilesearchstore-w6sbxr4fguzv/documents/sample-doc-fallback)  
**Excerpt**: and Entertainment P.Ltd
 Business Head

November 2010 November 2011 (1 year 1 month)

Silverscope Media and Entertainment P.Ltd is a subsidiary of Gee...
**Relevance**: Hig

In [ ]:
# Cell 5: Polished version with URI fallback, Markdown citations, and new query example
model = "gemini-2.5-flash"

config = types.GenerateContentConfig(
    tools=[
        types.Tool(
            file_search=types.FileSearch(
                file_search_store_names=[file_search_store.name]
            )
        )
    ]
)

# Try a slightly different query to demo variation
response = client.models.generate_content(
    model=model,
    contents="Did Lakshmi narayana work in Tesla?",
    config=config
)

# Display response with Markdown
display(Markdown(response.text))

# Polished: Print grounding as formatted citations
if response.candidates and len(response.candidates) > 0:
    candidate = response.candidates[0]
    if hasattr(candidate, 'grounding_metadata') and candidate.grounding_metadata:
        grounding = candidate.grounding_metadata

        # Collect unique chunks for citations
        chunks = grounding.grounding_chunks if grounding.grounding_chunks else []
        if chunks:
            print("\n📚 **Citations from Your Documents** (Grounded Sources):")
            seen_chunks = set()  # Avoid duplicates
            for chunk in chunks:
                if id(chunk) in seen_chunks:
                    continue
                seen_chunks.add(id(chunk))

                title = getattr(chunk.retrieved_context, 'title', 'Untitled')
                uri = getattr(chunk.retrieved_context, 'uri', None)
                if uri is None:
                    # Fallback: Construct from store (replace with real doc ID if known)
                    uri = f"{file_search_store.name}/documents/sample-doc-fallback"

                excerpt = chunk.retrieved_context.text[:150].strip() + "..." if len(chunk.retrieved_context.text) > 150 else chunk.retrieved_context.text.strip()

                print(f"\n[{title}]({uri})  \n**Excerpt**: {excerpt}")
                print(f"**Relevance**: High (full chunk retrieved for semantic match)")

            # Segments as inline response links
            if grounding.grounding_supports:
                print(f"\n🔗 **Grounded Parts of This Response** (Exact Matches):")
                for j, support in enumerate(grounding.grounding_supports):
                    chunk_idx = support.grounding_chunk_indices[0] if support.grounding_chunk_indices else 'N/A'
                    segment = support.segment.text.strip()
                    start, end = support.segment.start_index, support.segment.end_index
                    truncated = segment[:80] + "..." if len(segment) > 80 else segment
                    print(f"  {j+1}. **Chunk {chunk_idx}** → '{truncated}' (positions {start}-{end})")
        else:
            print("\nNo grounding chunks (pure generation—try a more file-specific query).")
    else:
        print("No grounding metadata.")
else:
    print("No response candidates.")

Lakshmi Narayana U's work experience, as detailed in the provided document, includes roles at Rachana Television Pvt. Ltd., Movisvami, Geetha Arts Digital, Cinematica Digitals Pvt. Ltd, Silverscope Media and Entertainment P.Ltd, Tech Mahindra (formerly Mahindra Satyam), Mahindra Satyam BPO, Catalytic Software, Arudra Interactive P.Ltd, Vortex Technologies, and Lanco Global Systems. There is no mention of Lakshmi Narayana U working at Tesla in the provided information.


📚 **Citations from Your Documents** (Grounded Sources):

[Sample Document](fileSearchStores/myfilesearchstore-w6sbxr4fguzv/documents/sample-doc-fallback)  
**Excerpt**: --- PAGE 1 ---

Contact

lakshyindy@gmail.com

www.linkedin.com/in/laksh

(LinkedIn)

Top Skills

Large Language Models (LLM)
 Agentic Al

Multi-agent...
**Relevance**: High (full chunk retrieved for semantic match)

[Sample Document](fileSearchStores/myfilesearchstore-w6sbxr4fguzv/documents/sample-doc-fallback)  
**Excerpt**: - February 2008 (1 year 5 months)

Head the Web 2.0 consulting practice with day-to-day project management
 responsibilities.

> Set up and drive the...
**Relevance**: High (full chunk retrieved for semantic match)

[Sample Document](fileSearchStores/myfilesearchstore-w6sbxr4fguzv/documents/sample-doc-fallback)  
**Excerpt**: and Entertainment P.Ltd
 Business Head

November 2010 November 2011 (1 year 1 month)

Silverscope Media and Entertainment P.Ltd is a subsidiary of Gee...
**Relevance**: High

In [ ]:
# Cell 5: Polished version with URI fallback, Markdown citations, and new query example
model = "gemini-2.5-flash"

config = types.GenerateContentConfig(
    tools=[
        types.Tool(
            file_search=types.FileSearch(
                file_search_store_names=[file_search_store.name]
            )
        )
    ]
)

# Try a slightly different query to demo variation
response = client.models.generate_content(
    model=model,
    contents="Did Lakshmi narayana work in Geetha Arts? What did he do there? Also what did he do before and after Geetha Arts",
    config=config
)

# Display response with Markdown
display(Markdown(response.text))

# Polished: Print grounding as formatted citations
if response.candidates and len(response.candidates) > 0:
    candidate = response.candidates[0]
    if hasattr(candidate, 'grounding_metadata') and candidate.grounding_metadata:
        grounding = candidate.grounding_metadata

        # Collect unique chunks for citations
        chunks = grounding.grounding_chunks if grounding.grounding_chunks else []
        if chunks:
            print("\n📚 **Citations from Your Documents** (Grounded Sources):")
            seen_chunks = set()  # Avoid duplicates
            for chunk in chunks:
                if id(chunk) in seen_chunks:
                    continue
                seen_chunks.add(id(chunk))

                title = getattr(chunk.retrieved_context, 'title', 'Untitled')
                uri = getattr(chunk.retrieved_context, 'uri', None)
                if uri is None:
                    # Fallback: Construct from store (replace with real doc ID if known)
                    uri = f"{file_search_store.name}/documents/sample-doc-fallback"

                excerpt = chunk.retrieved_context.text[:150].strip() + "..." if len(chunk.retrieved_context.text) > 150 else chunk.retrieved_context.text.strip()

                print(f"\n[{title}]({uri})  \n**Excerpt**: {excerpt}")
                print(f"**Relevance**: High (full chunk retrieved for semantic match)")

            # Segments as inline response links
            if grounding.grounding_supports:
                print(f"\n🔗 **Grounded Parts of This Response** (Exact Matches):")
                for j, support in enumerate(grounding.grounding_supports):
                    chunk_idx = support.grounding_chunk_indices[0] if support.grounding_chunk_indices else 'N/A'
                    segment = support.segment.text.strip()
                    start, end = support.segment.start_index, support.segment.end_index
                    truncated = segment[:80] + "..." if len(segment) > 80 else segment
                    print(f"  {j+1}. **Chunk {chunk_idx}** → '{truncated}' (positions {start}-{end})")
        else:
            print("\nNo grounding chunks (pure generation—try a more file-specific query).")
    else:
        print("No grounding metadata.")
else:
    print("No response candidates.")

Yes, Lakshmi Narayana U worked for companies that are subsidiaries of Geetha Arts. He served as the Business Head at Geetha Arts Digital from February 2010 to July 2015, at Cinematica Digitals Pvt. Ltd from June 2011 to February 2013, and at Silverscope Media and Entertainment P.Ltd from November 2010 to November 2011.

During his tenure at these Geetha Arts subsidiaries, he held various responsibilities:
*   **Geetha Arts Digital:** As Business Head, he focused on creating online, digital, and mobile distribution channels, managing partnerships with major market players like Amazon, iTunes, Netflix, and YouTube, developing and launching internet and mobile properties, initiating new ventures in overseas markets, driving enterprise-wide collaboration using tools like Yammer, and overseeing movie production and marketing partnerships.
*   **Cinematica Digitals Pvt. Ltd:** In his role as Business Head, he was responsible for driving and managing critical growth phases of the company and establishing and managing processes aligned with scale-up plans. Cinematica Digitals is the exclusive partner for Real Image's QUBE digital cinema solution in Andhra Pradesh.
*   **Silverscope Media and Entertainment P.Ltd:** He was involved in the overall management of the company, defining its strategy, business model, and market orientation. He also steered the company through management changes, stabilized operations, oversaw due-diligence, and concluded the sale of a majority stake.

Before his time with Geetha Arts' subsidiaries, Lakshmi Narayana had an extensive career including:
*   **Tech Mahindra (formerly Mahindra Satyam):** He worked as a Bid Director from August 2008 to August 2009.
*   **Mahindra Satyam BPO:** He was a Practice Head from March 2008 to August 2008.
*   **Catalytic Software:** He served as a Business Unit Head, leading the Web 2.0 consulting practice from October 2006 to February 2008.
*   **Arudra Interactive P.Ltd:** As Founder and CEO from 2002 to February 2007, he provided project management solutions for the entertainment industry, conceptualized and executed television and movie projects, introduced the concept of Computer Aided Process Management (CAPM) with the brand SHOWIZ, and developed Internet strategies for NGOs.
*   **Vortex Technologies:** He was a Director from 2004 to September 2006, contributing to consulting, delivery, business development, and product positioning in business performance management.
*   **Lanco Global Systems:** From April 1999 to November 2001, he held the position of Vice President, where he established Lanco's IT arm, advised the board on e-business strategy, and was responsible for launching several portals.
*   **IT Consultant:** From January 1997 to March 1999, he worked as a software consultant in the USA, focusing on Multimedia, Internet, and Groupware systems.

After his work with Geetha Arts' subsidiaries, Lakshmi Narayana continued his career with:
*   **Rachana Television Pvt. Ltd.:** From August 2015 to the present, he has been the Head of Digital Media, where he established and scaled the digital presence (YouTube, OTT platforms, Social Media, Mobile Apps) for channels like NTV, Bhakthi TV, and Vanitha TV, and is exploring Generative AI for content repurposing.
*   **Movisvami:** Since January 2022, he has been the Founder of Movisvami, a platform that connects cinema with business insights. He also developed Movisvami GPT, an AI-powered assistant, and authored the book "Directing Business: Management Lessons from Movie Dreamers."


📚 **Citations from Your Documents** (Grounded Sources):

[Sample Document](fileSearchStores/myfilesearchstore-w6sbxr4fguzv/documents/sample-doc-fallback)  
**Excerpt**: assistant that connects movie
 themes with real-world business and management lessons.

- Authored Directing Business: Management Lessons from Movie D...
**Relevance**: High (full chunk retrieved for semantic match)

[Sample Document](fileSearchStores/myfilesearchstore-w6sbxr4fguzv/documents/sample-doc-fallback)  
**Excerpt**: and Entertainment P.Ltd
 Business Head

November 2010 November 2011 (1 year 1 month)

Silverscope Media and Entertainment P.Ltd is a subsidiary of Gee...
**Relevance**: High (full chunk retrieved for semantic match)

[Sample Document](fileSearchStores/myfilesearchstore-w6sbxr4fguzv/documents/sample-doc-fallback)  
**Excerpt**: - February 2008 (1 year 5 months)

Head the Web 2.0 consulting practice with day-to-day project management
 responsibilities.

> Set up and drive the...
**Relevance**: High